# 🐍 Python & NumPy Fundamentals for Neuroscience

Welcome back, neural network explorer! 🧠⚡

Before we dive into the exciting world of brain signal analysis, let's make sure you're comfortable with the core tools you'll be using every day. This notebook is designed as a **refresher** - not a complete introduction to programming.

## What We'll Cover Today 🎯

1. **Python Essentials**: The language constructs you'll use constantly
2. **NumPy Fundamentals**: The backbone of scientific computing
3. **Data Structures**: Arrays, lists, and when to use each
4. **Mathematical Operations**: Linear algebra for neural networks
5. **Brain Signal Examples**: Real-world applications of what you're learning

## 💡 Why This Matters

In neuroscience, you'll constantly work with:
- **Multi-dimensional arrays** (time × channels × trials)
- **Mathematical operations** (filtering, Fourier transforms, matrix multiplication)
- **Data manipulation** (selecting time windows, averaging across trials)
- **Efficient computation** (vectorized operations for speed)

Master these fundamentals, and you'll breeze through the advanced topics!

---

*Ready to refresh your Python skills? Let's go! 🚀*

# 🛠️ Setting Up Our Environment

First, let's import the libraries we'll be using and set up some nice defaults for visualization.

In [ ]:
# Essential imports for today's session
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal
import time
import warnings
warnings.filterwarnings('ignore')

# Set up matplotlib for nice plots
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

# NumPy settings for cleaner output
np.set_printoptions(precision=3, suppress=True)

print("✅ Environment ready!")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")

# 🐍 Python Essentials: Quick Review

Let's quickly review the Python constructs you'll use most often in neuroscience computing.

## List Comprehensions: Your Secret Weapon 🔧

List comprehensions are incredibly useful for data processing. In neuroscience, you'll often need to apply operations across multiple files, channels, or trials.

In [ ]:
# Example: Processing multiple EEG channels
channel_names = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2']

# Traditional approach
frontal_channels = []
for channel in channel_names:
    if channel.startswith('F'):
        frontal_channels.append(channel)
        
print("Traditional approach:", frontal_channels)

# List comprehension (much cleaner!)
frontal_channels_lc = [ch for ch in channel_names if ch.startswith('F')]
print("List comprehension:", frontal_channels_lc)

# More complex example: Extract channel indices
frontal_indices = [i for i, ch in enumerate(channel_names) if ch.startswith('F')]
print("Frontal channel indices:", frontal_indices)

# With transformation
channel_pairs = [(ch, f"{ch}_processed") for ch in channel_names[:3]]
print("Channel pairs:", channel_pairs)

## Functions: Building Reusable Analysis Tools 🔨

In [ ]:
def calculate_power_spectrum(signal_data, sampling_rate=250, freq_bands=None):
    """
    Calculate power spectrum for EEG-like data.
    
    Parameters:
    -----------
    signal_data : array-like
        Time series data
    sampling_rate : int
        Sampling frequency in Hz
    freq_bands : dict, optional
        Dictionary of frequency bands to analyze
        
    Returns:
    --------
    frequencies : array
        Frequency values
    power : array
        Power spectral density
    """
    if freq_bands is None:
        freq_bands = {
            'delta': (0.5, 4),
            'theta': (4, 8),
            'alpha': (8, 13),
            'beta': (13, 30)
        }
    
    # Calculate power spectrum using Welch's method
    frequencies, power = signal.welch(signal_data, sampling_rate, nperseg=sampling_rate*2)
    
    # Calculate band powers
    band_powers = {}
    for band, (low, high) in freq_bands.items():
        band_mask = (frequencies >= low) & (frequencies <= high)
        band_powers[band] = np.trapz(power[band_mask], frequencies[band_mask])
    
    return frequencies, power, band_powers

# Test our function
test_signal = np.random.randn(1000) + 2*np.sin(2*np.pi*10*np.linspace(0, 4, 1000))
freqs, psd, bands = calculate_power_spectrum(test_signal)

print("Band powers:")
for band, power in bands.items():
    print(f"  {band}: {power:.3f}")

## Classes: Organizing Complex Analysis Pipelines 🏗️

In [ ]:
class EEGProcessor:
    """A simple EEG processing class to demonstrate object-oriented concepts."""
    
    def __init__(self, sampling_rate=250, channels=None):
        self.sampling_rate = sampling_rate
        self.channels = channels or ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4']
        self.processed_data = None
        
    def load_data(self, data):
        """Load EEG data (channels × time_points)."""
        self.raw_data = np.array(data)
        print(f"Data loaded: {self.raw_data.shape} (channels × time_points)")
        
    def apply_bandpass_filter(self, low_freq=1, high_freq=40):
        """Apply a bandpass filter to remove noise."""
        if self.raw_data is None:
            raise ValueError("No data loaded! Use load_data() first.")
            
        # Design butterworth filter
        sos = signal.butter(4, [low_freq, high_freq], 
                           btype='bandpass', fs=self.sampling_rate, output='sos')
        
        # Apply filter to each channel
        self.processed_data = np.zeros_like(self.raw_data)
        for i, channel_data in enumerate(self.raw_data):
            self.processed_data[i] = signal.sosfilt(sos, channel_data)
            
        print(f"✅ Bandpass filter applied ({low_freq}-{high_freq} Hz)")
        
    def plot_channel(self, channel_idx=0, duration=2.0):
        """Plot raw vs processed data for a specific channel."""
        if self.raw_data is None:
            raise ValueError("No data loaded!")
            
        # Time axis
        n_samples = int(duration * self.sampling_rate)
        time_axis = np.linspace(0, duration, n_samples)
        
        plt.figure(figsize=(12, 6))
        
        # Plot raw data
        plt.subplot(2, 1, 1)
        plt.plot(time_axis, self.raw_data[channel_idx, :n_samples], 'b-', alpha=0.7)
        plt.title(f'Raw EEG - Channel {self.channels[channel_idx]}')
        plt.ylabel('Amplitude (μV)')
        plt.grid(True, alpha=0.3)
        
        # Plot processed data (if available)
        if self.processed_data is not None:
            plt.subplot(2, 1, 2)
            plt.plot(time_axis, self.processed_data[channel_idx, :n_samples], 'r-', alpha=0.7)
            plt.title(f'Filtered EEG - Channel {self.channels[channel_idx]}')
            plt.ylabel('Amplitude (μV)')
            plt.xlabel('Time (seconds)')
            plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Demo the class
processor = EEGProcessor(sampling_rate=250)

# Simulate some EEG data with noise
time_points = np.linspace(0, 10, 2500)  # 10 seconds at 250 Hz
n_channels = len(processor.channels)

# Create realistic EEG-like signals
fake_eeg = np.zeros((n_channels, len(time_points)))
for i in range(n_channels):
    # Alpha waves (10 Hz) + noise + some artifacts
    alpha_wave = 2 * np.sin(2 * np.pi * 10 * time_points + i * 0.5)
    theta_wave = 1 * np.sin(2 * np.pi * 6 * time_points + i * 0.3)
    noise = 0.5 * np.random.randn(len(time_points))
    artifacts = 10 * np.sin(2 * np.pi * 60 * time_points)  # 60 Hz line noise
    
    fake_eeg[i] = alpha_wave + theta_wave + noise + artifacts

# Process the data
processor.load_data(fake_eeg)
processor.apply_bandpass_filter(low_freq=1, high_freq=40)
processor.plot_channel(channel_idx=0, duration=3.0)

# 🔢 NumPy Fundamentals: The Heart of Scientific Computing

NumPy is the foundation of scientific computing in Python. In neuroscience, you'll use it for everything from basic array operations to complex signal processing.

## Array Creation and Basic Operations 🎯

In [ ]:
# Different ways to create arrays (common in neuroscience)

# 1. Time axis for signals
sampling_rate = 250  # Hz
duration = 2.0  # seconds
time = np.linspace(0, duration, int(sampling_rate * duration))
print(f"Time axis: {time[:5]}...{time[-5:]}")
print(f"Shape: {time.shape}")

# 2. Initialize arrays for data storage
n_channels = 64
n_timepoints = 1000
n_trials = 100

# Empty array for EEG data (channels × time × trials)
eeg_data = np.zeros((n_channels, n_timepoints, n_trials))
print(f"\nEEG data array shape: {eeg_data.shape}")

# Random data (useful for testing)
random_signals = np.random.randn(n_channels, n_timepoints)
print(f"Random signals shape: {random_signals.shape}")

# 3. Structured arrays for experiments
trial_conditions = np.array(['rest', 'task', 'rest', 'task'] * 25)
print(f"\nTrial conditions (first 10): {trial_conditions[:10]}")

# 4. Frequency arrays for spectral analysis
frequencies = np.fft.fftfreq(n_timepoints, 1/sampling_rate)
positive_freqs = frequencies[:n_timepoints//2]
print(f"\nFrequency range: {positive_freqs[0]:.1f} to {positive_freqs[-1]:.1f} Hz")

## Array Indexing and Slicing: Extracting What You Need 🎯

In neuroscience, you'll constantly need to extract specific time windows, channels, or trials.

In [ ]:
# Create sample EEG data (channels × time × trials)
np.random.seed(42)  # For reproducible results
n_channels, n_timepoints, n_trials = 10, 1000, 50
eeg_data = np.random.randn(n_channels, n_timepoints, n_trials)

# Add some structure to make it more realistic
time_axis = np.linspace(0, 4, n_timepoints)  # 4 seconds
for ch in range(n_channels):
    for trial in range(n_trials):
        # Add alpha waves (10 Hz) with some variability
        alpha_freq = 10 + np.random.normal(0, 1)
        eeg_data[ch, :, trial] += 2 * np.sin(2 * np.pi * alpha_freq * time_axis)

print(f"EEG data shape: {eeg_data.shape}")
print(f"Data type: {eeg_data.dtype}")
print(f"Memory usage: {eeg_data.nbytes / 1024:.1f} KB")

# Common indexing operations
print("\n=== Common Indexing Operations ===")

# 1. Select specific channels
frontal_channels = [0, 1, 2]  # Assuming these are frontal
frontal_data = eeg_data[frontal_channels, :, :]
print(f"1. Frontal channels data shape: {frontal_data.shape}")

# 2. Select time window (e.g., 1-3 seconds)
start_time, end_time = 1.0, 3.0
start_idx = int(start_time * (n_timepoints / 4))  # 4 seconds total
end_idx = int(end_time * (n_timepoints / 4))
time_window = eeg_data[:, start_idx:end_idx, :]
print(f"2. Time window (1-3s) shape: {time_window.shape}")

# 3. Select specific trials
task_trials = [1, 3, 5, 7, 9]  # Assuming these are task trials
task_data = eeg_data[:, :, task_trials]
print(f"3. Task trials data shape: {task_data.shape}")

# 4. Complex indexing: frontal channels, specific time window, task trials
subset = eeg_data[frontal_channels, start_idx:end_idx, task_trials]
print(f"4. Complex subset shape: {subset.shape}")

# 5. Boolean indexing (very powerful!)
# Find time points where signal is above threshold
channel_0_trial_0 = eeg_data[0, :, 0]
high_amplitude_mask = np.abs(channel_0_trial_0) > 2
high_amplitude_points = channel_0_trial_0[high_amplitude_mask]
print(f"5. High amplitude points: {len(high_amplitude_points)} out of {n_timepoints}")

# 6. Advanced: condition-based selection
# Get trials where the mean amplitude is above median
trial_means = np.mean(eeg_data, axis=(0, 1))  # Mean across channels and time
high_activity_trials = trial_means > np.median(trial_means)
high_activity_data = eeg_data[:, :, high_activity_trials]
print(f"6. High activity trials: {high_activity_data.shape[2]} out of {n_trials}")

## Array Operations: The Power of Vectorization ⚡

Vectorized operations are much faster than Python loops. This is crucial when processing large neuroscience datasets.

In [ ]:
# Let's compare vectorized vs loop operations
n_samples = 100000
data = np.random.randn(n_samples)

# Method 1: Python loop (slow)
start_time = time.time()
result_loop = []
for value in data:
    result_loop.append(value ** 2 + 2 * value + 1)
loop_time = time.time() - start_time

# Method 2: Vectorized operations (fast)
start_time = time.time()
result_vectorized = data ** 2 + 2 * data + 1
vectorized_time = time.time() - start_time

print(f"Loop method: {loop_time:.4f} seconds")
print(f"Vectorized method: {vectorized_time:.4f} seconds")
print(f"Speedup: {loop_time/vectorized_time:.1f}x faster!")

# Verify they give the same result
print(f"Results match: {np.allclose(result_loop, result_vectorized)}")

## Mathematical Operations for Neuroscience 🧮

Let's explore the mathematical operations you'll use most frequently in brain signal analysis.

In [ ]:
# Create sample multi-channel EEG data
np.random.seed(123)
n_channels, n_timepoints, n_trials = 8, 500, 20
eeg_data = np.random.randn(n_channels, n_timepoints, n_trials)

# Add realistic signal structure
time = np.linspace(0, 2, n_timepoints)  # 2 seconds
for ch in range(n_channels):
    for trial in range(n_trials):
        # Add alpha rhythm (10 Hz) and some theta (6 Hz)
        alpha = 3 * np.sin(2 * np.pi * 10 * time + ch * 0.2)
        theta = 1.5 * np.sin(2 * np.pi * 6 * time + ch * 0.1)
        eeg_data[ch, :, trial] += alpha + theta

print(f"Sample EEG data shape: {eeg_data.shape}")
print(f"Data range: {eeg_data.min():.2f} to {eeg_data.max():.2f}")

# === Statistical Operations ===
print("\n=== Statistical Operations ===")

# 1. Basic statistics across different axes
trial_averages = np.mean(eeg_data, axis=2)  # Average across trials
channel_averages = np.mean(eeg_data, axis=0)  # Average across channels
time_averages = np.mean(eeg_data, axis=1)  # Average across time

print(f"Trial averages shape: {trial_averages.shape}")
print(f"Channel averages shape: {channel_averages.shape}")
print(f"Time averages shape: {time_averages.shape}")

# 2. Standard deviation and variance
trial_std = np.std(eeg_data, axis=2)
print(f"\nStandard deviation across trials: {trial_std.mean():.3f}")

# 3. Percentiles (useful for outlier detection)
percentiles = np.percentile(eeg_data, [5, 25, 50, 75, 95])
print(f"Data percentiles: {percentiles}")

# === Signal Processing Operations ===
print("\n=== Signal Processing Operations ===")

# 1. Z-score normalization (common preprocessing step)
eeg_normalized = (eeg_data - np.mean(eeg_data, axis=1, keepdims=True)) / np.std(eeg_data, axis=1, keepdims=True)
print(f"Normalized data mean: {np.mean(eeg_normalized):.6f}")
print(f"Normalized data std: {np.std(eeg_normalized):.6f}")

# 2. Baseline correction (subtract pre-stimulus period)
baseline_period = slice(0, 50)  # First 50 time points
baseline_mean = np.mean(eeg_data[:, baseline_period, :], axis=1, keepdims=True)
eeg_baseline_corrected = eeg_data - baseline_mean
print(f"Baseline corrected data range: {eeg_baseline_corrected.min():.2f} to {eeg_baseline_corrected.max():.2f}")

# 3. Root Mean Square (RMS) - measure of signal power
rms_values = np.sqrt(np.mean(eeg_data**2, axis=1))
print(f"RMS values shape: {rms_values.shape}")
print(f"Average RMS across channels: {np.mean(rms_values):.3f}")

# === Advanced Operations ===
print("\n=== Advanced Operations ===")

# 1. Correlation between channels
# Take first trial, correlate channels across time
channel_correlations = np.corrcoef(eeg_data[:, :, 0])
print(f"Channel correlation matrix shape: {channel_correlations.shape}")
print(f"Average correlation: {np.mean(channel_correlations[np.triu_indices(n_channels, k=1)]):.3f}")

# 2. Covariance matrix
cov_matrix = np.cov(eeg_data[:, :, 0])
print(f"Covariance matrix shape: {cov_matrix.shape}")

# 3. Principal Component Analysis (PCA) preparation
# Center the data
data_centered = eeg_data - np.mean(eeg_data, axis=1, keepdims=True)
# Compute covariance matrix
cov_temporal = np.cov(data_centered[:, :, 0].T)  # Time x Time covariance
eigenvalues, eigenvectors = np.linalg.eig(cov_temporal)
print(f"Top 5 eigenvalues: {np.sort(eigenvalues)[-5:][::-1]}")

# 4. Sliding window analysis
window_size = 50
step_size = 10
windows = []
for start in range(0, n_timepoints - window_size, step_size):
    window_data = eeg_data[:, start:start+window_size, :]
    window_mean = np.mean(window_data, axis=1)  # Average across time in window
    windows.append(window_mean)

windowed_analysis = np.array(windows)
print(f"Windowed analysis shape: {windowed_analysis.shape}")
print(f"Number of windows: {len(windows)}")

## Linear Algebra for Neural Networks 🧠

Understanding matrix operations is crucial for neural networks and many signal processing techniques.

In [ ]:
# === Matrix Operations Fundamental to Neural Networks ===
print("=== Matrix Operations for Neural Networks ===")

# 1. Basic matrix multiplication (the heart of neural networks)
# Simulate a simple neural network layer
input_features = 64  # e.g., 64 EEG channels
hidden_units = 32
batch_size = 10

# Input data (batch_size × input_features)
X = np.random.randn(batch_size, input_features)
print(f"Input X shape: {X.shape}")

# Weight matrix (input_features × hidden_units)
W = np.random.randn(input_features, hidden_units) * 0.1  # Small random weights
print(f"Weight W shape: {W.shape}")

# Bias vector (hidden_units,)
b = np.zeros(hidden_units)
print(f"Bias b shape: {b.shape}")

# Forward pass: linear transformation
z = np.dot(X, W) + b  # or X @ W + b
print(f"Linear output z shape: {z.shape}")

# Activation function (ReLU)
a = np.maximum(0, z)
print(f"Activated output a shape: {a.shape}")
print(f"Activated units: {np.sum(a > 0)} out of {a.size}")

# 2. Batch operations (processing multiple samples simultaneously)
print("\n=== Batch Operations ===")

# Simulate processing multiple EEG epochs
n_epochs = 100
n_channels = 10
n_timepoints = 250

# Create batch of EEG data
eeg_batch = np.random.randn(n_epochs, n_channels, n_timepoints)
print(f"EEG batch shape: {eeg_batch.shape}")

# Flatten each epoch for neural network input
eeg_flattened = eeg_batch.reshape(n_epochs, -1)
print(f"Flattened EEG shape: {eeg_flattened.shape}")

# Apply transformation to entire batch
feature_weights = np.random.randn(n_channels * n_timepoints, 50)
features = np.dot(eeg_flattened, feature_weights)
print(f"Extracted features shape: {features.shape}")

# 3. Covariance and correlation matrices (important for connectivity analysis)
print("\n=== Covariance and Correlation ===")

# Simulate multi-channel EEG data
n_channels = 5
n_timepoints = 1000
np.random.seed(42)

# Create correlated channels (simulate brain connectivity)
base_signal = np.random.randn(n_timepoints)
channels = np.zeros((n_channels, n_timepoints))

for i in range(n_channels):
    # Each channel is base signal + independent noise
    channels[i] = 0.7 * base_signal + 0.3 * np.random.randn(n_timepoints)

# Compute correlation matrix
correlation_matrix = np.corrcoef(channels)
print(f"Correlation matrix shape: {correlation_matrix.shape}")
print(f"Correlation matrix:")
print(correlation_matrix)

# Compute covariance matrix
covariance_matrix = np.cov(channels)
print(f"\nCovariance matrix shape: {covariance_matrix.shape}")

# 4. Eigendecomposition (used in PCA, ICA, etc.)
print("\n=== Eigendecomposition ===")

eigenvalues, eigenvectors = np.linalg.eig(correlation_matrix)
print(f"Eigenvalues: {eigenvalues}")
print(f"Eigenvectors shape: {eigenvectors.shape}")

# Sort by eigenvalue magnitude
idx = np.argsort(eigenvalues)[::-1]
eigenvalues_sorted = eigenvalues[idx]
eigenvectors_sorted = eigenvectors[:, idx]

print(f"Sorted eigenvalues: {eigenvalues_sorted}")
print(f"Explained variance ratio: {eigenvalues_sorted / np.sum(eigenvalues_sorted)}")

# 5. Matrix norms (useful for regularization)
print("\n=== Matrix Norms ===")

sample_matrix = np.random.randn(5, 5)
print(f"Frobenius norm: {np.linalg.norm(sample_matrix, 'fro'):.3f}")
print(f"Nuclear norm: {np.linalg.norm(sample_matrix, 'nuc'):.3f}")
print(f"Spectral norm: {np.linalg.norm(sample_matrix, 2):.3f}")

# 6. Solving linear systems (used in many algorithms)
print("\n=== Solving Linear Systems ===")

# Example: least squares solution
A = np.random.randn(100, 10)  # Design matrix
x_true = np.random.randn(10)  # True parameters
y = A @ x_true + 0.1 * np.random.randn(100)  # Noisy observations

# Solve least squares: x = (A^T A)^(-1) A^T y
x_estimated = np.linalg.solve(A.T @ A, A.T @ y)
print(f"True parameters: {x_true[:5]}")
print(f"Estimated parameters: {x_estimated[:5]}")
print(f"Estimation error: {np.linalg.norm(x_true - x_estimated):.6f}")

# 🧬 Real-World Example: EEG Signal Analysis

Let's put everything together with a realistic neuroscience example: analyzing EEG signals to detect different brain states.

In [ ]:
# === Simulate Realistic EEG Data ===
print("=== Simulating Realistic EEG Data ===")

# Parameters
sampling_rate = 250  # Hz
duration = 10  # seconds
n_channels = 8  # Simplified EEG montage
n_trials = 60  # 30 eyes-closed, 30 eyes-open

# Channel names (simplified 10-20 system)
channel_names = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4']

# Time axis
time = np.linspace(0, duration, int(sampling_rate * duration))
n_timepoints = len(time)

# Initialize data array
eeg_data = np.zeros((n_channels, n_timepoints, n_trials))
conditions = ['eyes_closed'] * 30 + ['eyes_open'] * 30

# Generate realistic EEG signals
np.random.seed(42)

for trial in range(n_trials):
    condition = conditions[trial]
    
    for ch in range(n_channels):
        # Base noise
        signal = 0.5 * np.random.randn(n_timepoints)
        
        # Add physiological rhythms
        if condition == 'eyes_closed':
            # Strong alpha rhythm (8-12 Hz) especially in posterior channels
            if 'P' in channel_names[ch]:  # Posterior channels
                alpha_power = 4.0
            else:
                alpha_power = 2.0
            alpha_freq = 10 + np.random.normal(0, 0.5)
            signal += alpha_power * np.sin(2 * np.pi * alpha_freq * time + np.random.uniform(0, 2*np.pi))
            
            # Some theta (4-8 Hz)
            theta_freq = 6 + np.random.normal(0, 0.5)
            signal += 1.0 * np.sin(2 * np.pi * theta_freq * time + np.random.uniform(0, 2*np.pi))
            
        else:  # eyes_open
            # Reduced alpha, more beta activity (13-30 Hz)
            alpha_power = 1.0
            alpha_freq = 10 + np.random.normal(0, 0.5)
            signal += alpha_power * np.sin(2 * np.pi * alpha_freq * time + np.random.uniform(0, 2*np.pi))
            
            # Beta activity
            beta_freq = 20 + np.random.normal(0, 2)
            signal += 1.5 * np.sin(2 * np.pi * beta_freq * time + np.random.uniform(0, 2*np.pi))
            
            # Higher frequency noise (more alertness)
            signal += 0.3 * np.random.randn(n_timepoints)
        
        # Add some 60Hz line noise
        signal += 0.1 * np.sin(2 * np.pi * 60 * time)
        
        # Store the signal
        eeg_data[ch, :, trial] = signal

print(f"Generated EEG data shape: {eeg_data.shape}")
print(f"Conditions: {len(set(conditions))} unique conditions")
print(f"Data range: {eeg_data.min():.2f} to {eeg_data.max():.2f} μV")

In [ ]:
# === Exploratory Data Analysis ===
print("=== Exploratory Data Analysis ===")

# 1. Visualize sample trials
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot sample eyes-closed trial
axes[0, 0].plot(time[:1000], eeg_data[6, :1000, 0])  # P3 channel, first 4 seconds
axes[0, 0].set_title('Eyes Closed - P3 Channel')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Amplitude (μV)')
axes[0, 0].grid(True, alpha=0.3)

# Plot sample eyes-open trial
axes[0, 1].plot(time[:1000], eeg_data[6, :1000, 35])  # P3 channel, eyes-open trial
axes[0, 1].set_title('Eyes Open - P3 Channel')
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Amplitude (μV)')
axes[0, 1].grid(True, alpha=0.3)

# 2. Power spectral density comparison
from scipy.signal import welch

# Compute PSD for both conditions
freqs, psd_closed = welch(eeg_data[6, :, :30].mean(axis=1), sampling_rate, nperseg=sampling_rate*2)
freqs, psd_open = welch(eeg_data[6, :, 30:].mean(axis=1), sampling_rate, nperseg=sampling_rate*2)

axes[1, 0].semilogy(freqs, psd_closed, label='Eyes Closed', alpha=0.8)
axes[1, 0].semilogy(freqs, psd_open, label='Eyes Open', alpha=0.8)
axes[1, 0].set_xlim(0, 40)
axes[1, 0].set_xlabel('Frequency (Hz)')
axes[1, 0].set_ylabel('PSD (μV²/Hz)')
axes[1, 0].set_title('Power Spectral Density - P3 Channel')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 3. Topographic map of alpha power
alpha_band = (8, 12)
alpha_indices = (freqs >= alpha_band[0]) & (freqs <= alpha_band[1])

alpha_power_closed = np.zeros(n_channels)
alpha_power_open = np.zeros(n_channels)

for ch in range(n_channels):
    # Eyes closed
    freqs_ch, psd_ch = welch(eeg_data[ch, :, :30].mean(axis=1), sampling_rate, nperseg=sampling_rate*2)
    alpha_power_closed[ch] = np.trapz(psd_ch[alpha_indices], freqs_ch[alpha_indices])
    
    # Eyes open
    freqs_ch, psd_ch = welch(eeg_data[ch, :, 30:].mean(axis=1), sampling_rate, nperseg=sampling_rate*2)
    alpha_power_open[ch] = np.trapz(psd_ch[alpha_indices], freqs_ch[alpha_indices])

# Bar plot of alpha power by channel
x = np.arange(n_channels)
width = 0.35

axes[1, 1].bar(x - width/2, alpha_power_closed, width, label='Eyes Closed', alpha=0.8)
axes[1, 1].bar(x + width/2, alpha_power_open, width, label='Eyes Open', alpha=0.8)
axes[1, 1].set_xlabel('Channel')
axes[1, 1].set_ylabel('Alpha Power (μV²)')
axes[1, 1].set_title('Alpha Power by Channel (8-12 Hz)')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(channel_names, rotation=45)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# === Statistical Analysis ===
print("\n=== Statistical Analysis ===")

# Compare alpha power between conditions
print("Alpha power comparison:")
print(f"Eyes Closed - Mean: {alpha_power_closed.mean():.3f}, Std: {alpha_power_closed.std():.3f}")
print(f"Eyes Open - Mean: {alpha_power_open.mean():.3f}, Std: {alpha_power_open.std():.3f}")

# Effect size (Cohen's d)
pooled_std = np.sqrt(((alpha_power_closed.std()**2) + (alpha_power_open.std()**2)) / 2)
cohens_d = (alpha_power_closed.mean() - alpha_power_open.mean()) / pooled_std
print(f"Effect size (Cohen's d): {cohens_d:.3f}")

# Find channels with largest differences
alpha_diff = alpha_power_closed - alpha_power_open
max_diff_channel = np.argmax(alpha_diff)
print(f"\nLargest alpha difference in channel: {channel_names[max_diff_channel]} ({alpha_diff[max_diff_channel]:.3f} μV²)")

# Correlation between channels
print("\n=== Channel Connectivity Analysis ===")

# Compute average correlation matrices for each condition
corr_closed = np.zeros((n_channels, n_channels))
corr_open = np.zeros((n_channels, n_channels))

for trial in range(30):
    corr_closed += np.corrcoef(eeg_data[:, :, trial])
    corr_open += np.corrcoef(eeg_data[:, :, trial + 30])

corr_closed /= 30
corr_open /= 30

# Plot correlation matrices
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

im1 = axes[0].imshow(corr_closed, cmap='coolwarm', vmin=-1, vmax=1)
axes[0].set_title('Eyes Closed - Correlation Matrix')
axes[0].set_xticks(range(n_channels))
axes[0].set_yticks(range(n_channels))
axes[0].set_xticklabels(channel_names, rotation=45)
axes[0].set_yticklabels(channel_names)
plt.colorbar(im1, ax=axes[0])

im2 = axes[1].imshow(corr_open, cmap='coolwarm', vmin=-1, vmax=1)
axes[1].set_title('Eyes Open - Correlation Matrix')
axes[1].set_xticks(range(n_channels))
axes[1].set_yticks(range(n_channels))
axes[1].set_xticklabels(channel_names, rotation=45)
axes[1].set_yticklabels(channel_names)
plt.colorbar(im2, ax=axes[1])

# Difference matrix
corr_diff = corr_closed - corr_open
im3 = axes[2].imshow(corr_diff, cmap='RdBu_r', vmin=-0.5, vmax=0.5)
axes[2].set_title('Correlation Difference (Closed - Open)')
axes[2].set_xticks(range(n_channels))
axes[2].set_yticks(range(n_channels))
axes[2].set_xticklabels(channel_names, rotation=45)
axes[2].set_yticklabels(channel_names)
plt.colorbar(im3, ax=axes[2])

plt.tight_layout()
plt.show()

# Summary statistics
avg_corr_closed = np.mean(corr_closed[np.triu_indices(n_channels, k=1)])
avg_corr_open = np.mean(corr_open[np.triu_indices(n_channels, k=1)])

print(f"Average correlation - Eyes Closed: {avg_corr_closed:.3f}")
print(f"Average correlation - Eyes Open: {avg_corr_open:.3f}")
print(f"Difference: {avg_corr_closed - avg_corr_open:.3f}")

# 🏋️ Practice Exercises: Test Your Skills!

Time to put your NumPy skills to the test! Try these exercises that mirror real neuroscience challenges.

## Exercise 1: Event-Related Potential (ERP) Analysis 🎯

**Task**: Analyze event-related potentials by extracting epochs around stimulus events and computing averaged responses.

**Background**: In EEG experiments, we often want to see how the brain responds to specific stimuli (like seeing a face or hearing a tone). We extract small time windows around each stimulus and average across trials to reveal the typical brain response.

In [ ]:
# === Exercise 1: Event-Related Potential Analysis ===
print("=== Exercise 1: ERP Analysis ===")

# Simulate continuous EEG data
sampling_rate = 250  # Hz
duration = 120  # seconds (2 minutes)
n_channels = 4
channel_names = ['Fz', 'Cz', 'Pz', 'Oz']

# Generate continuous EEG
np.random.seed(123)
time_continuous = np.linspace(0, duration, int(sampling_rate * duration))
continuous_eeg = np.random.randn(n_channels, len(time_continuous)) * 2

# Add some background alpha rhythm
for ch in range(n_channels):
    continuous_eeg[ch] += 3 * np.sin(2 * np.pi * 10 * time_continuous + ch * 0.5)

# Generate stimulus events (random times)
n_events = 50
event_times = np.sort(np.random.uniform(5, duration-5, n_events))  # Avoid edges
event_samples = (event_times * sampling_rate).astype(int)

# Add stimulus-locked responses to the data
for event_sample in event_samples:
    # Create a stereotypical ERP response
    erp_time = np.linspace(0, 1, 250)  # 1 second response
    
    # N100 component (negative deflection at 100ms)
    n100 = -5 * np.exp(-((erp_time - 0.1)**2) / (2 * 0.02**2))
    
    # P300 component (positive deflection at 300ms)
    p300 = 8 * np.exp(-((erp_time - 0.3)**2) / (2 * 0.05**2))
    
    erp_response = n100 + p300
    
    # Add to each channel (with some variability)
    for ch in range(n_channels):
        if event_sample + 250 < continuous_eeg.shape[1]:  # Make sure we don't go out of bounds
            # Different channels have different sensitivities
            sensitivity = [0.5, 1.0, 1.2, 0.8][ch]  # Cz and Pz are most sensitive
            continuous_eeg[ch, event_sample:event_sample+250] += sensitivity * erp_response

print(f"Continuous EEG shape: {continuous_eeg.shape}")
print(f"Number of events: {n_events}")
print(f"Event times (first 10): {event_times[:10]}")

# YOUR TASK: Complete the following functions

def extract_epochs(continuous_data, event_samples, pre_stim=0.2, post_stim=0.8, sampling_rate=250):
    """
    Extract epochs around stimulus events.
    
    Parameters:
    -----------
    continuous_data : array, shape (n_channels, n_timepoints)
        Continuous EEG data
    event_samples : array
        Sample indices of stimulus events
    pre_stim : float
        Time before stimulus (seconds)
    post_stim : float
        Time after stimulus (seconds)
    sampling_rate : int
        Sampling frequency
        
    Returns:
    --------
    epochs : array, shape (n_channels, n_timepoints_epoch, n_events)
        Extracted epochs
    epoch_times : array
        Time axis for epochs (relative to stimulus)
    """
    # TODO: Implement epoch extraction
    pre_samples = int(pre_stim * sampling_rate)
    post_samples = int(post_stim * sampling_rate)
    epoch_length = pre_samples + post_samples
    
    n_channels = continuous_data.shape[0]
    valid_events = []
    epochs_list = []
    
    for event_sample in event_samples:
        start_sample = event_sample - pre_samples
        end_sample = event_sample + post_samples
        
        # Check if epoch is within data bounds
        if start_sample >= 0 and end_sample < continuous_data.shape[1]:
            epoch = continuous_data[:, start_sample:end_sample]
            epochs_list.append(epoch)
            valid_events.append(event_sample)
    
    epochs = np.stack(epochs_list, axis=2)
    epoch_times = np.linspace(-pre_stim, post_stim, epoch_length)
    
    return epochs, epoch_times

def baseline_correct_epochs(epochs, baseline_period, epoch_times):
    """
    Apply baseline correction to epochs.
    
    Parameters:
    -----------
    epochs : array, shape (n_channels, n_timepoints, n_events)
        Epoch data
    baseline_period : tuple
        (start_time, end_time) for baseline period
    epoch_times : array
        Time axis for epochs
        
    Returns:
    --------
    epochs_corrected : array
        Baseline-corrected epochs
    """
    # TODO: Implement baseline correction
    baseline_mask = (epoch_times >= baseline_period[0]) & (epoch_times <= baseline_period[1])
    baseline_mean = np.mean(epochs[:, baseline_mask, :], axis=1, keepdims=True)
    epochs_corrected = epochs - baseline_mean
    
    return epochs_corrected

def compute_erp(epochs):
    """
    Compute event-related potential by averaging across trials.
    
    Parameters:
    -----------
    epochs : array, shape (n_channels, n_timepoints, n_events)
        Epoch data
        
    Returns:
    --------
    erp : array, shape (n_channels, n_timepoints)
        Averaged ERP
    erp_std : array, shape (n_channels, n_timepoints)
        Standard deviation across trials
    """
    # TODO: Implement ERP computation
    erp = np.mean(epochs, axis=2)
    erp_std = np.std(epochs, axis=2)
    
    return erp, erp_std

# Test your implementation
epochs, epoch_times = extract_epochs(continuous_eeg, event_samples, pre_stim=0.2, post_stim=0.8)
epochs_corrected = baseline_correct_epochs(epochs, baseline_period=(-0.2, 0), epoch_times=epoch_times)
erp, erp_std = compute_erp(epochs_corrected)

print(f"\nResults:")
print(f"Epochs shape: {epochs.shape}")
print(f"ERP shape: {erp.shape}")
print(f"Epoch time range: {epoch_times[0]:.3f} to {epoch_times[-1]:.3f} seconds")

# Plot results
plt.figure(figsize=(12, 8))

for ch in range(n_channels):
    plt.subplot(2, 2, ch + 1)
    plt.plot(epoch_times, erp[ch], 'b-', linewidth=2, label='ERP')
    plt.fill_between(epoch_times, erp[ch] - erp_std[ch], erp[ch] + erp_std[ch], 
                     alpha=0.3, color='blue', label='±1 SD')
    plt.axvline(0, color='red', linestyle='--', alpha=0.7, label='Stimulus')
    plt.axhline(0, color='black', linestyle='-', alpha=0.3)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude (μV)')
    plt.title(f'ERP - Channel {channel_names[ch]}')
    plt.grid(True, alpha=0.3)
    if ch == 0:
        plt.legend()

plt.tight_layout()
plt.show()

# Analysis questions for you to think about:
print("\n=== Analysis Questions ===")
print("1. Which channel shows the strongest P300 response?")
print("2. At what time does the N100 component peak?")
print("3. How does the signal-to-noise ratio compare across channels?")

# Find P300 peak
p300_window = (epoch_times >= 0.2) & (epoch_times <= 0.4)
p300_peaks = np.argmax(erp[:, p300_window], axis=1)
p300_peak_times = epoch_times[p300_window][p300_peaks]

print(f"\nP300 peak times by channel:")
for ch in range(n_channels):
    print(f"  {channel_names[ch]}: {p300_peak_times[ch]:.3f} s")

## Exercise 2: Spectral Analysis and Connectivity 🌊

**Task**: Analyze the frequency content of neural signals and compute connectivity between brain regions.

**Background**: The brain operates at different frequency bands (delta, theta, alpha, beta, gamma). Understanding how these frequencies change and how they're synchronized between brain regions gives us insights into brain function.

In [ ]:
# === Exercise 2: Spectral Analysis and Connectivity ===
print("=== Exercise 2: Spectral Analysis and Connectivity ===")

# Generate sample data with known connectivity
np.random.seed(456)
sampling_rate = 500  # Hz
duration = 60  # seconds
n_channels = 6
channel_names = ['F3', 'F4', 'C3', 'C4', 'P3', 'P4']

time = np.linspace(0, duration, int(sampling_rate * duration))
n_timepoints = len(time)

# Create signals with known connectivity patterns
signals = np.zeros((n_channels, n_timepoints))

# Generate base oscillations
alpha_source = np.sin(2 * np.pi * 10 * time)  # 10 Hz alpha
beta_source = np.sin(2 * np.pi * 20 * time)   # 20 Hz beta
gamma_source = np.sin(2 * np.pi * 40 * time)  # 40 Hz gamma

# Add noise and connectivity patterns
for ch in range(n_channels):
    # Base noise
    signals[ch] = 0.5 * np.random.randn(n_timepoints)
    
    # Add frequency-specific connectivity
    if 'F' in channel_names[ch]:  # Frontal channels
        signals[ch] += 2 * beta_source + 0.5 * np.random.randn(n_timepoints)
    elif 'C' in channel_names[ch]:  # Central channels
        signals[ch] += 1.5 * alpha_source + 1 * beta_source + 0.5 * np.random.randn(n_timepoints)
    elif 'P' in channel_names[ch]:  # Posterior channels
        signals[ch] += 3 * alpha_source + 0.5 * np.random.randn(n_timepoints)
    
    # Add some gamma coupling
    if ch % 2 == 0:  # Left hemisphere
        signals[ch] += 0.8 * gamma_source
    else:  # Right hemisphere
        # Delayed gamma (simulate inter-hemispheric delay)
        delay_samples = int(0.01 * sampling_rate)  # 10ms delay
        delayed_gamma = np.zeros_like(gamma_source)
        delayed_gamma[delay_samples:] = gamma_source[:-delay_samples]
        signals[ch] += 0.8 * delayed_gamma

print(f"Generated signals shape: {signals.shape}")
print(f"Duration: {duration} seconds")
print(f"Sampling rate: {sampling_rate} Hz")

# YOUR TASK: Implement the following functions

def compute_power_spectrum(data, sampling_rate, nperseg=None):
    """
    Compute power spectral density for multi-channel data.
    
    Parameters:
    -----------
    data : array, shape (n_channels, n_timepoints)
        Multi-channel signal data
    sampling_rate : int
        Sampling frequency
    nperseg : int, optional
        Length of each segment for Welch's method
        
    Returns:
    --------
    frequencies : array
        Frequency values
    psd : array, shape (n_channels, n_frequencies)
        Power spectral density for each channel
    """
    # TODO: Implement power spectrum computation
    if nperseg is None:
        nperseg = sampling_rate * 4  # 4 second windows
    
    n_channels = data.shape[0]
    
    # Compute PSD for first channel to get frequency array
    frequencies, psd_ch = signal.welch(data[0], sampling_rate, nperseg=nperseg)
    
    # Initialize PSD array
    psd = np.zeros((n_channels, len(frequencies)))
    
    # Compute PSD for each channel
    for ch in range(n_channels):
        frequencies, psd[ch] = signal.welch(data[ch], sampling_rate, nperseg=nperseg)
    
    return frequencies, psd

def compute_band_power(psd, frequencies, freq_bands):
    """
    Compute power in specific frequency bands.
    
    Parameters:
    -----------
    psd : array, shape (n_channels, n_frequencies)
        Power spectral density
    frequencies : array
        Frequency values
    freq_bands : dict
        Dictionary of frequency bands {band_name: (low_freq, high_freq)}
        
    Returns:
    --------
    band_powers : dict
        Dictionary of band powers {band_name: array of powers per channel}
    """
    # TODO: Implement band power computation
    band_powers = {}
    
    for band_name, (low_freq, high_freq) in freq_bands.items():
        # Create frequency mask
        freq_mask = (frequencies >= low_freq) & (frequencies <= high_freq)
        
        # Compute power in band for each channel
        band_power = np.zeros(psd.shape[0])
        for ch in range(psd.shape[0]):
            band_power[ch] = np.trapz(psd[ch, freq_mask], frequencies[freq_mask])
        
        band_powers[band_name] = band_power
    
    return band_powers

def compute_coherence(data, sampling_rate, nperseg=None):
    """
    Compute coherence between all pairs of channels.
    
    Parameters:
    -----------
    data : array, shape (n_channels, n_timepoints)
        Multi-channel signal data
    sampling_rate : int
        Sampling frequency
    nperseg : int, optional
        Length of each segment
        
    Returns:
    --------
    frequencies : array
        Frequency values
    coherence : array, shape (n_channels, n_channels, n_frequencies)
        Coherence matrix
    """
    # TODO: Implement coherence computation
    if nperseg is None:
        nperseg = sampling_rate * 4
    
    n_channels = data.shape[0]
    
    # Compute coherence for first pair to get frequency array
    frequencies, coh_temp = signal.coherence(data[0], data[1], sampling_rate, nperseg=nperseg)
    
    # Initialize coherence array
    coherence = np.zeros((n_channels, n_channels, len(frequencies)))
    
    # Compute coherence for all pairs
    for i in range(n_channels):
        for j in range(n_channels):
            if i == j:
                coherence[i, j, :] = 1.0  # Perfect coherence with self
            else:
                frequencies, coherence[i, j, :] = signal.coherence(data[i], data[j], sampling_rate, nperseg=nperseg)
    
    return frequencies, coherence

# Test your implementation
frequencies, psd = compute_power_spectrum(signals, sampling_rate)

freq_bands = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 80)
}

band_powers = compute_band_power(psd, frequencies, freq_bands)
coh_frequencies, coherence = compute_coherence(signals, sampling_rate)

print(f"\nResults:")
print(f"PSD shape: {psd.shape}")
print(f"Frequency range: {frequencies[0]:.1f} to {frequencies[-1]:.1f} Hz")
print(f"Coherence shape: {coherence.shape}")

# Plot results
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Power spectra
for ch in range(n_channels):
    axes[0, 0].semilogy(frequencies, psd[ch], label=channel_names[ch], alpha=0.8)
axes[0, 0].set_xlim(0, 60)
axes[0, 0].set_xlabel('Frequency (Hz)')
axes[0, 0].set_ylabel('PSD (μV²/Hz)')
axes[0, 0].set_title('Power Spectral Density')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Band powers
band_names = list(freq_bands.keys())
x_pos = np.arange(len(band_names))
bar_width = 0.1

for ch in range(n_channels):
    powers = [band_powers[band][ch] for band in band_names]
    axes[0, 1].bar(x_pos + ch * bar_width, powers, bar_width, 
                   label=channel_names[ch], alpha=0.8)

axes[0, 1].set_xlabel('Frequency Band')
axes[0, 1].set_ylabel('Power (μV²)')
axes[0, 1].set_title('Band Powers')
axes[0, 1].set_xticks(x_pos + bar_width * 2.5)
axes[0, 1].set_xticklabels(band_names)
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Coherence heatmap (average across frequencies)
avg_coherence = np.mean(coherence, axis=2)
im = axes[0, 2].imshow(avg_coherence, cmap='viridis', vmin=0, vmax=1)
axes[0, 2].set_title('Average Coherence Matrix')
axes[0, 2].set_xticks(range(n_channels))
axes[0, 2].set_yticks(range(n_channels))
axes[0, 2].set_xticklabels(channel_names)
axes[0, 2].set_yticklabels(channel_names)
plt.colorbar(im, ax=axes[0, 2])

# 4. Alpha coherence network
alpha_idx = (coh_frequencies >= 8) & (coh_frequencies <= 13)
alpha_coherence = np.mean(coherence[:, :, alpha_idx], axis=2)
im2 = axes[1, 0].imshow(alpha_coherence, cmap='viridis', vmin=0, vmax=1)
axes[1, 0].set_title('Alpha Band Coherence (8-13 Hz)')
axes[1, 0].set_xticks(range(n_channels))
axes[1, 0].set_yticks(range(n_channels))
axes[1, 0].set_xticklabels(channel_names)
axes[1, 0].set_yticklabels(channel_names)
plt.colorbar(im2, ax=axes[1, 0])

# 5. Beta coherence network
beta_idx = (coh_frequencies >= 13) & (coh_frequencies <= 30)
beta_coherence = np.mean(coherence[:, :, beta_idx], axis=2)
im3 = axes[1, 1].imshow(beta_coherence, cmap='viridis', vmin=0, vmax=1)
axes[1, 1].set_title('Beta Band Coherence (13-30 Hz)')
axes[1, 1].set_xticks(range(n_channels))
axes[1, 1].set_yticks(range(n_channels))
axes[1, 1].set_xticklabels(channel_names)
axes[1, 1].set_yticklabels(channel_names)
plt.colorbar(im3, ax=axes[1, 1])

# 6. Coherence spectrum between F3 and F4
f3_idx = channel_names.index('F3')
f4_idx = channel_names.index('F4')
axes[1, 2].plot(coh_frequencies, coherence[f3_idx, f4_idx, :], 'b-', linewidth=2)
axes[1, 2].set_xlim(0, 60)
axes[1, 2].set_ylim(0, 1)
axes[1, 2].set_xlabel('Frequency (Hz)')
axes[1, 2].set_ylabel('Coherence')
axes[1, 2].set_title('F3-F4 Coherence Spectrum')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Analysis summary
print("\n=== Analysis Summary ===")
print("Band power analysis:")
for band in band_names:
    max_channel = np.argmax(band_powers[band])
    print(f"  {band}: strongest in {channel_names[max_channel]} ({band_powers[band][max_channel]:.3f} μV²)")

print("\nCoherence analysis:")
# Find strongest coherence pairs (excluding self-connections)
coherence_no_diag = avg_coherence.copy()
np.fill_diagonal(coherence_no_diag, 0)
max_coherence_idx = np.unravel_index(np.argmax(coherence_no_diag), coherence_no_diag.shape)
print(f"  Strongest coherence: {channel_names[max_coherence_idx[0]]}-{channel_names[max_coherence_idx[1]]} ({coherence_no_diag[max_coherence_idx]:.3f})")

# Interhemispheric coherence
left_channels = [i for i, ch in enumerate(channel_names) if '3' in ch]
right_channels = [i for i, ch in enumerate(channel_names) if '4' in ch]
interhemispheric_coherence = []

for l_ch in left_channels:
    for r_ch in right_channels:
        if channel_names[l_ch][0] == channel_names[r_ch][0]:  # Same region
            coh_val = avg_coherence[l_ch, r_ch]
            interhemispheric_coherence.append(coh_val)
            print(f"  {channel_names[l_ch]}-{channel_names[r_ch]} coherence: {coh_val:.3f}")

print(f"  Average interhemispheric coherence: {np.mean(interhemispheric_coherence):.3f}")

# 🎯 Key Takeaways: What You've Learned

Congratulations! You've just completed a comprehensive review of Python and NumPy fundamentals for neuroscience. Here's what you've mastered:

## 🐍 Python Skills
- **List comprehensions** for efficient data processing
- **Functions** for building reusable analysis tools
- **Classes** for organizing complex analysis pipelines
- **Best practices** for scientific computing

## 🔢 NumPy Mastery
- **Array creation and manipulation** for multi-dimensional brain data
- **Indexing and slicing** to extract specific channels, time windows, and trials
- **Vectorized operations** for lightning-fast computations
- **Mathematical operations** essential for signal processing
- **Linear algebra** foundations for neural networks

## 🧠 Neuroscience Applications
- **EEG data processing** with realistic multi-channel signals
- **Event-related potential (ERP) analysis** for studying brain responses
- **Spectral analysis** to understand frequency content
- **Connectivity analysis** using coherence measures
- **Statistical analysis** of neural data

## 💡 Performance Tips You've Learned

1. **Always use vectorized operations** instead of Python loops
2. **Specify axis parameters** carefully when working with multi-dimensional data
3. **Use keepdims=True** when you need to preserve array dimensions
4. **Leverage NumPy's broadcasting** for efficient operations
5. **Choose appropriate data types** to save memory

---

## 🚀 What's Next?

In the next notebook (`02_ml_foundations.ipynb`), you'll build on these fundamentals to explore:

- **Machine learning concepts** relevant to neuroscience
- **Classification and regression** for brain state detection
- **Cross-validation** and model evaluation
- **Feature engineering** for neural signals
- **Preparing for deep learning** with PyTorch

You're well-equipped for this journey! The NumPy skills you've practiced here will be your foundation for everything that comes next.

---

**Ready to continue your deep learning journey? See you in the next notebook! 🧠⚡**